In [1]:
import tools.processing as pre
import re

# use less text for now to avoid memory error
text = pre.get_text("data/cleaned-rap-lyrics/clean2_pac_.txt")

TIMESTEPS = 6

vocab = pre.Vocabulary(text)

# double \\n to avoid null error in tensorboard projection
text = text.replace("\n", " \\n ")

# remove extra spacing
tokens = re.sub( " +", " ", text).split(" ")[:-1]


str_data, str_labels = pre.create_data_label_pairs(tokens, TIMESTEPS)

print( list( zip(str_data, str_labels) )[:5] )

[(['as', 'real', 'as', 'it', 'seems', 'the'], 'american'), (['real', 'as', 'it', 'seems', 'the', 'american'], 'dream'), (['as', 'it', 'seems', 'the', 'american', 'dream'], '\\n'), (['it', 'seems', 'the', 'american', 'dream', '\\n'], "ain't"), (['seems', 'the', 'american', 'dream', '\\n', "ain't"], 'nothing')]


In [2]:
import tools.training as tr

encoder = tr.IndexWordEncoder("Index-Word-Encoding", vocab.word2index)
decoder = tr.OneHotWordDecoder("1-Hot-Word-Decoding", vocab.index2word, temperature=0.8)

data = encoder.encode( str_data )
labels = encoder.encode_labels( str_labels )

my_seed = " ".join(str_data[0]).replace("\\n", "\n")

del str_data, str_labels

In [3]:
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

EPOCHS = 12
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)

Building model from scratch! 
 Saving into: '../logs/train_model'


Epoch 1/12
Loss:    	 5.62436580657959
Accuracy:	 13.877730369567871
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the kuwait minute 
 my still levi's want of all 
 
 dank the hit i bed my who's 
 what's now i be what can me is 
 grew phone we kiddin' of goin' done girl 
 to the turned back 
 couldn't it impossible posse 
 that got like


Epoch 2/12
Loss:    	 5.1255598068237305
Accuracy:	 17.610109329223633
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the harmed 
 gorden the excel the motherfuckers 
 keeping hell gift with nothin' living somewhere got a day 
 defeat you i and a not now that i if are and don't so i thought and moocher 
 to the gat the hat to get 
 me no to the


Epoch 3/12
Loss:    	 4.40452241897583
Accuracy:	 23.248849868774414
------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the b

In [4]:
decoder.temperature = 0.9

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=40)

sampler("as real as it seems the")

------Sampling----------
seed: 
as real as it seems the
-
result: 
as real as it seems the american dream though ain't nothing but it seems no with the rhymes to five niggaz on the block 
 through still they got me over 
 man i up the shit now up back in the smoker 
 they told


In [56]:
decoder = tr.MaxOneHotDecoder("1-Hot-Word-Decoding", vocab.index2word)

sampler = lambda seed_text: tr.sample( seed_text, rnn, encoder, decoder, length=40)

sampler("as real as it seems the")

AttributeError: module 'tools.training' has no attribute 'MaxOneHotDecoder'

In [ ]:
import tools.architectures as nn

HIDDEN_LAYER_SIZE = 512
VOCAB_SIZE = len(vocab.word2index)

TIMESTEPS = 6
EPOCHS = 20
BATCH_SIZE = 256

EMBEDDING_SIZE = 256

rnn = nn.EmbeddedSingleLayerRNN(name = "multi-rakim")
rnn.build(HIDDEN_LAYER_SIZE, VOCAB_SIZE, EMBEDDING_SIZE, TIMESTEPS, l2_reg=0.0)

sampler = lambda trainable, _: tr.sample( my_seed, trainable, encoder, decoder, length=50)

tr.train_model(rnn, data, labels, sampler, epochs=EPOCHS, batch_size=BATCH_SIZE)